In [2]:
import pandas as pd
import os
import sys
from pathlib import Path
from typing import Optional

# Add project root to path to import utilities
project_root = Path("..")
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.utils.data_loader import snake_case, sanitize_currency_series, normalize_chunk

def load_procurement_data(csv_path: str = "../data/data.csv", sample_size: Optional[int] = None) -> pd.DataFrame:
    """
    Load the California procurement dataset for analysis.
    
    Args:
        csv_path: Path to the CSV file (relative to notebooks folder)
        sample_size: Number of rows to load (None for all data)
    
    Returns:
        Normalized pandas DataFrame
    """
    print(f"Loading data from {csv_path}...")
    
    # Read CSV with optional sampling
    if sample_size:
        df = pd.read_csv(csv_path, nrows=sample_size)
        print(f"Loaded sample of {len(df)} rows")
    else:
        df = pd.read_csv(csv_path)
        print(f"Loaded {len(df)} rows")
    
    # Apply same normalization as the application
    records = list(normalize_chunk(df))
    df_normalized = pd.DataFrame(records)
    
    # Convert date strings to datetime objects for easier analysis
    date_columns = ['creation_date', 'purchase_date']
    for col in date_columns:
        if col in df_normalized.columns:
            df_normalized[col] = pd.to_datetime(df_normalized[col], format='%m/%d/%Y', errors='coerce')
    
    print(f"Data normalized with {len(df_normalized)} rows and {len(df_normalized.columns)} columns")
    return df_normalized

# Load the data
df = load_procurement_data()

Loading data from ../data/data.csv...
Loaded 346018 rows
Data normalized with 346018 rows and 31 columns


## EASY Queries (Basic filtering and counting)

### Query 1: How many total purchase orders are there?
**Complexity**: Easy - Simple counting operation  
**Why easy**: Basic aggregation, single operation, no complex filtering or grouping

**Natural Language**: "Show me the total number of purchase orders in the dataset"

**Pandas Code**:

In [3]:
# Count unique purchase orders (since multiple rows can share same PO number)
total_orders = df['purchase_order_number'].nunique()
print(f"Total unique purchase orders: {total_orders}")

Total unique purchase orders: 200533


### Query 2: What are the different fiscal years in the data?
**Complexity**: Easy - Basic distinct values  
**Why easy**: Simple field extraction, no calculations or complex logic

**Natural Language**: "What fiscal years does this dataset cover?"

**Pandas Code**:

In [4]:
# Get unique fiscal years
fiscal_years = df['fiscal_year'].unique()
print("Fiscal years in dataset:", sorted(fiscal_years))

Fiscal years in dataset: ['2012-2013', '2013-2014', '2014-2015']


### Query 3: Which departments are the most common?
**Complexity**: Easy - Basic grouping and counting  
**Why easy**: Simple groupby operation, straightforward aggregation

**Natural Language**: "Show me the top 5 departments by number of orders"

**Pandas Code**:


In [6]:
# Top departments by order count
top_departments = (df.groupby('department_name')['purchase_order_number']
                     .nunique()                  # <-- this is the key
                     .sort_values(ascending=False)
                     .head(5))
print(top_departments)

department_name
Corrections and Rehabilitation, Department of    36040
Correctional Health Care Services                28549
Transportation, Department of                    14871
Forestry and Fire Protection, Department of      14546
State Hospitals, Department of                    8457
Name: purchase_order_number, dtype: int64


## MEDIUM Queries (Filtering, calculations, and multi-step operations)

### Query 4: What's the total spending by fiscal year?
**Complexity**: Medium - Aggregation with filtering  
**Why medium**: Requires grouping by time period, sum calculation, and proper data types

**Natural Language**: "Calculate the total procurement spend for each fiscal year"

**Pandas Code**:



In [16]:
# Total spend by fiscal year
spend_by_year = df.groupby('fiscal_year')['total_price'].sum().sort_values(ascending=False)
print("Total spend by fiscal year:")
print(spend_by_year.apply(lambda x: f"${x:,.2f}"))

Total spend by fiscal year:
fiscal_year
2012-2013    $61,912,420,939.42
2014-2015    $46,860,483,903.82
2013-2014    $42,470,675,654.96
Name: total_price, dtype: object


### Query 5: What are the most expensive orders?
**Complexity**: Medium - Sorting and limiting with multiple fields  
**Why medium**: Requires sorting, selecting specific columns, and understanding which fields to display

**Natural Language**: "Show me the top 10 most expensive purchase orders with supplier and department details"

**Pandas Code**:



In [17]:
# Top 10 most expensive orders
top_expensive = df.nlargest(10, 'total_price')[['purchase_order_number', 'supplier_name', 'department_name', 'total_price', 'fiscal_year']]
print("Top 10 most expensive orders:")
print(top_expensive.to_string(index=False))

Top 10 most expensive orders:
purchase_order_number                        supplier_name                     department_name  total_price fiscal_year
             11-88134           Delta Dental of California Health Care Services, Department of 7337038064.0   2012-2013
             04-36069                L.A. Care Health Plan Health Care Services, Department of 3194190000.0   2012-2013
             12-89371                County of Los Angeles Health Care Services, Department of 3010052803.0   2013-2014
             03-76182 Health Net Community Solutions, Inc. Health Care Services, Department of 2474118000.0   2013-2014
             03-76182 Health Net Community Solutions, Inc. Health Care Services, Department of 2253227000.0   2013-2014
             11-10019             Ramsell Public Health Rx        Public Health, Department of 2200000000.0   2012-2013
             10-87128                       Ventura County Health Care Services, Department of 1979109000.0   2014-2015
          

### Query 6: How many orders were created in 2014?
**Complexity**: Medium - Date filtering  
**Why medium**: Requires date parsing and filtering, understanding date formats

**Natural Language**: "Count how many purchase orders were created in calendar year 2014"

**Pandas Code**:




In [18]:
# Orders created in 2014
orders_2014 = df[df['creation_date'].dt.year == 2014]['purchase_order_number'].nunique()
print(f"Unique purchase orders created in 2014: {orders_2014}")

Unique purchase orders created in 2014: 70100


### Query 7: What's the average order value by acquisition type?
**Complexity**: Medium - Multi-level aggregation  
**Why medium**: Group by categorical field, calculate averages, handle missing data

**Natural Language**: "What are the average order values for different types of acquisitions?"

**Pandas Code**:

In [19]:
# Average order value by acquisition type
avg_by_acquisition = df.groupby('acquisition_type').agg({
    'total_price': 'mean',
    'purchase_order_number': 'count'
}).round(2)

avg_by_acquisition.columns = ['avg_order_value', 'order_count']
avg_by_acquisition = avg_by_acquisition.sort_values('avg_order_value', ascending=False)
print("Average order value by acquisition type:")
print(avg_by_acquisition)

Average order value by acquisition type:
                       avg_order_value  order_count
acquisition_type                                   
NON-IT Services             2056842.48        68372
IT Services                  387407.41        11516
IT Telecommunications         92560.41          147
IT Goods                      30937.21        50900
NON-IT Goods                  21220.13       215083


## HARD Queries (Complex filtering, multiple aggregations, date calculations)

### Query 8: Monthly spending trends for 2014
**Complexity**: Hard - Time-series analysis with date extraction  
**Why hard**: Requires date parsing, month extraction, time-based grouping, trend analysis

**Natural Language**: "Show me the monthly procurement spending trends for calendar year 2014"

**Pandas Code**:

In [24]:
# Monthly spending trends for 2014
df_2014 = df[df['creation_date'].dt.year == 2014].copy()
df_2014['month'] = df_2014['creation_date'].dt.month

monthly_spend = df_2014.groupby('month')['total_price'].sum()
monthly_spend.index = pd.to_datetime(monthly_spend.index, format='%m').strftime('%B')

print("Monthly spending in 2014:")
for month, spend in monthly_spend.items():
    print(f"{month}: ${spend:,.2f}")

Monthly spending in 2014:
January: $521,996,259.94
February: $4,302,116,700.37
March: $5,293,888,708.42
April: $3,422,321,431.35
May: $1,158,778,421.46
June: $11,037,274,239.93
July: $2,495,931,781.95
August: $1,920,683,472.45
September: $1,521,035,169.36
October: $1,249,028,210.39
November: $1,741,325,210.62
December: $838,374,106.56


### Query 9: Top suppliers by total spend in each fiscal year
**Complexity**: Hard - Multi-dimensional grouping with ranking  
**Why hard**: Requires grouping by multiple fields, ranking within groups, complex reshaping

**Natural Language**: "Who are the top 3 suppliers by total spend in each fiscal year?"

**Pandas Code**:

In [7]:
# Top 3 suppliers by total spend per fiscal year
top_suppliers_by_year = (
    df.groupby(['fiscal_year', 'supplier_name'], as_index=False)['total_price']
    .sum()
)

# Correct ranking: use 'min' so ties share the same rank and we don't skip
top_suppliers_by_year['rank'] = top_suppliers_by_year.groupby('fiscal_year')['total_price'] \
    .rank(method='min', ascending=False)

# Keep all suppliers with rank <= 3 (this correctly includes ties)
top_3_per_year = top_suppliers_by_year[top_suppliers_by_year['rank'] <= 3] \
    .sort_values(['fiscal_year', 'rank', 'total_price'], ascending=[True, True, False])

# Pretty print
print("Top 3 suppliers by total spend in each fiscal year:\n")
for year in sorted(top_3_per_year['fiscal_year'].unique()):
    print(f"{year}:")
    year_data = top_3_per_year[top_3_per_year['fiscal_year'] == year]
    for _, row in year_data.iterrows():
        print(f"  {int(row['rank'])}. {row['supplier_name']}: ${row['total_price']:,.2f}")
    print()

Top 3 suppliers by total spend in each fiscal year:

2012-2013:
  1. L.A. Care Health Plan: $11,160,129,000.04
  2. Delta Dental of California: $8,172,038,064.03
  3. Inland Empire Health Plan: $5,413,666,000.03

2013-2014:
  1. Health Net Community Solutions, Inc.: $9,461,076,000.04
  2. County of Los Angeles: $3,220,301,231.42
  3. Blue Cross of California Partnership Plan, Inc.: $2,357,953,000.06

2014-2015:
  1. Fresno-Kings-Madera Regional Health Authority: $3,509,792,000.00
  2. Unknown: $2,993,076,155.07
  3. Ventura County: $2,907,422,778.10

